In [ ]:
import sys
import os
import collections
import json
from itertools import cycle
from ast import literal_eval
from dataclasses import dataclass, asdict
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm, trange

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch import optim
from torch.optim import lr_scheduler
import torchmetrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import *
from transformers.modeling_outputs import SequenceClassifierOutput, ModelOutput

import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'PAN SCD with contrastive learning.ipynb'
import wandb

In [ ]:
result_folder = os.environ["scratch_result_folder"] if "scratch_result_folder" in os.environ else '../result'
scratch_data_folder = os.environ["scratch_data_folder"] if "scratch_data_folder" in os.environ else None
repo_folder = os.environ["style_models_repo_folder"] if "style_models_repo_folder" in os.environ else None
data_folder = f"{repo_folder}/data" if repo_folder else '../data'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# https://github.com/huggingface/transformers/issues/5486
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
@dataclass
class MyTrainingArgs:
    # dataset args # it's not very appropriate to put them here, especially the split
    dataset_idx: int
#     split: str
        
    # model args
    base_model_name: str ='bert-base-uncased'
    freeze_bert: bool = 0
    use_pooler: bool = False
        
    # training args    
    num_epoch: int = 5
    lr: float = 5e-5
    num_warmup_steps = 500
    warmup_ratio = 0.1
    model_folder: str = None # if None, this will be inferred based on tasks
    model_name: str = None # if provide, use to name model_folder, otherwise use style to name model_folder
    loss_fn: str = None
    do_mlm: bool = False
    # tempurature for cosine similarity. 
    # simcse uses 0.05. However, that's for crossentropy on single label classification. 
    # For multi-label where BCEloss(withlogits) is used, due to sigmoid, temp < 1 tends to make it learn slower
    cos_temp: float = 1. 
        
    # data loader args
    batch_size: int = 32
    max_length: int = 64
    shuffle: bool = False
    num_workers: int = 4
    data_limit: int = None # if not None, truncate dataset to keep only top {data_limit} rows
    
    # post training args
    save_best_only: bool = True
    load_best_at_end: bool = True
    early_stop_patience: int = 1
    
    def __post_init__(self):
        excute_time = datetime.now() 
        model_name = self.model_name if self.model_name else f"pan22-dataset{self.dataset_idx}"
        model_folder = f"{result_folder}/{model_name}/{excute_time.now().strftime('%Y%m%d-%H:%M:%S')}"
        self.model_folder = model_folder

In [ ]:
class Pan22Datasets(Dataset): 
    # currently it's a Mapping-style dataset. Not sure if a Iterable-style dataset will be better
    # this works for standard class indices and also class probilities
    # limit: use to truncate dataset. This will drop rows after certain index. May influence label distribution.
    def __init__(self, dataset_idx, split, training_args):
        self.dataset_idx = dataset_idx # [1,2,3]. Pan22 only has these three.
        self.split = split
        self.max_length = training_args.max_length
        self.dataset_folder = f"{data_folder}/pan22/dataset{self.dataset_idx}/{self.split}"
        self.num_docs = len(os.listdir(self.dataset_folder))//2 if self.split != 'test' else len(os.listdir(self.dataset_folder))
#         self.tokenizer = AutoTokenizer.from_pretrained(training_args.base_model_name)
        
    def __len__(self):
        return self.num_docs

    def __getitem__(self, idx): 
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        text_file = f"problem-{idx+1}.txt"
        truth_file = f"truth-problem-{idx+1}.json"
        with open(f"{self.dataset_folder}/{text_file}") as f:
            doc = f.readlines()
        with open(f"{self.dataset_folder}/{truth_file}") as f:
            truth = json.load(f)
            
        item = {'paragraph_authors':truth['paragraph-authors'], 'changes':truth['changes'], 'text': doc}
        
        return item

In [ ]:
def collate_fn(batch):
    '''
    This is only for batchsize==1
    '''
    batch_out = batch[0]
    
    batch_out.update({k:v for k,v in tokenizer(text = batch_out['text'], return_tensors='pt', padding=True, truncation=True, max_length=my_training_args.max_length).to(device).items()})
    
    del batch_out['text']
    return batch_out

In [ ]:
class Similarity(nn.Module):
    """
    Dot product or cosine similarity
    From SimCSE
    """

    def __init__(self, temp):
        super().__init__()
        self.temp = temp
        self.cos = nn.CosineSimilarity(dim=-1)

    def forward(self, x, y):
        return self.cos(x, y) / self.temp

In [ ]:
@dataclass
class SCDOutput(ModelOutput):
    loss: torch.FloatTensor = None
    cos_sim: torch.FloatTensor = None
    sent_embs: List[torch.FloatTensor] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
class BertForSCD(BertPreTrainedModel):
    def __init__(self, config, training_args):
        super().__init__(config)
        self.use_pooler = training_args.use_pooler
        self.basemodel = AutoModel.from_pretrained(training_args.base_model_name)
        self.do_mlm = training_args.do_mlm
        self.cossim = Similarity(training_args.cos_temp)
        if training_args.loss_fn == 'BCEWithLogitsLoss':
            self.loss_fn = nn.BCEWithLogitsLoss()
        elif training_args.loss_fn == 'MSELoss':
            self.loss_fn = nn.MSELoss()
        
        # mlm is not finished yet
#         if self.do_mlm: 
#             self.lm_head = BertLMPredictionHead(config)

    def forward(self, input_ids, token_type_ids, attention_mask, output_sent_embs=False, output_hidden_states=False, output_attentions=False, **kwargs):
        output1 = self.basemodel(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, output_hidden_states=output_hidden_states, output_attentions=output_attentions)
        output2 = self.basemodel(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        
        if self.use_pooler and ('pooler_output' in output):
            sent_emb1 = output1['pooler_output']
            sent_emb2 = output2['pooler_output']
        else:
            sent_emb1 = output1['last_hidden_state'][:,0,:]
            sent_emb2 = output2['last_hidden_state'][:,0,:]
        
        # get contrastive_labels
        num_paras = len(kwargs['paragraph_authors'])
        contrastive_labels = torch.zeros(num_paras,num_paras)
        paragraphs_in_group = collections.defaultdict(list)
        for i in range(num_paras):
            paragraphs_in_group[kwargs['paragraph_authors'][i]].append(i)
        for v in paragraphs_in_group.values():
            i_grid,j_grid=torch.meshgrid(torch.tensor(v),torch.tensor(v), indexing='ij')
            contrastive_labels[i_grid,j_grid] = 1.
        contrastive_labels = contrastive_labels.to(device)
        
        cos_sim = self.cossim(sent_emb1.unsqueeze(1), sent_emb2.unsqueeze(0))
        loss = self.loss_fn(cos_sim, contrastive_labels)
        if output_sent_embs:
            sent_embs = [sent_emb1.detach(), sent_emb2.detach()]
        else:
            sent_embs = None
        
        return SCDOutput(loss=loss, cos_sim=cos_sim, sent_embs=sent_embs, hidden_states=output1.hidden_states, attentions=output1.attentions)
        

In [ ]:
def do_eval(model, cycle_valid_loader, num_valid_steps):
    '''
    cos_sim gives similarity
    style change is the opposite of it
    '''
    
    model.eval()
    
    threshold = 0.5 if my_training_args.loss_fn == 'MSELoss' else 0.0
    
    f1 = torchmetrics.F1Score(num_classes=2, average='macro')
    
    for i_step in trange(num_valid_steps, leave=False):
        batch = next(cycle_valid_loader)
        output = model(**batch)
        
        prediction = (torch.diag(output.cos_sim.detach().cpu(),-1)<threshold).type(torch.int64)   
        style_change_labels = torch.tensor(batch['changes'])
        f1.update(prediction, style_change_labels)
        
    evaluation = {'f1': f1.compute().item()}
    return evaluation

In [ ]:
def init_model(training_args):
    config = AutoConfig.from_pretrained(training_args.base_model_name) 
    model = BertForSCD(config, training_args).to(device)
    return model

In [ ]:
def freeze_model(model, freeze_bert):
    '''
    if freeze_bert == True, freeze all layer. 
    if freeze_bert is a positive integer, freeze the bottom {freeze_bert} attention layers
    negative integer should also work
    '''
    if freeze_bert==True:
        for param in model.basemodel.parameters():
            param.requires_grad = False
    elif isinstance(freeze_bert, (int, np.int32, np.int64, torch.int32, torch.int64)):
        for param in model.basemodel.embeddings.parameters():
            param.requires_grad = False  
        for layer in model.basemodel.encoder.layer[:freeze_bert]: 
            for param in layer.parameters():
                param.requires_grad = False  
    return model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') 

# run

In [ ]:
num_epochs = 20
batch_size = 1
max_length = 128
save_best_only = True # if not save the best, save the last
monitering_metric = 'f1'

early_stop_patience = 5

my_training_args = MyTrainingArgs(dataset_idx=1,
                                  model_name=f'PAN SCD with contrastive learning',
                                  base_model_name='bert-base-uncased',
                                  freeze_bert=False, # will be overwriten in the loop
                                  use_pooler=False,
                                  num_epoch=num_epochs,
                                  batch_size=batch_size,
                                  max_length=max_length,
                                  save_best_only=save_best_only,
                                  early_stop_patience=early_stop_patience,
                                 )

# some parameters to sweep
LR = [3e-5, 5e-5, 7e-5]
FREEZE_BERT = [9, 11]
WARMUP_RATIO = [0.1, 0.15, 0.2]
LOSS_FN = ['BCEWithLogitsLoss', 'MSELoss']

LR, FREEZE_BERT, WARMUP_RATIO, LOSS_FN = np.meshgrid(LR,FREEZE_BERT, WARMUP_RATIO, LOSS_FN)
LR, FREEZE_BERT, WARMUP_RATIO, LOSS_FN = LR.flatten(), FREEZE_BERT.flatten(), WARMUP_RATIO.flatten(), LOSS_FN.flatten()
num_runs = len(LR)

# create dataset
train_set = Pan22Datasets(dataset_idx=1, split='train', training_args=my_training_args)
val_set = Pan22Datasets(dataset_idx=1, split='validation', training_args=my_training_args)
train_loader = DataLoader(train_set, batch_size=1, shuffle=my_training_args.shuffle, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=1, shuffle=my_training_args.shuffle, collate_fn=collate_fn)

num_training_steps = len(train_loader)
cycle_train_loader = cycle(iter(train_loader))

num_valid_steps = len(val_loader)
cycle_val_loader = cycle(iter(val_loader))

# start runs
for i_run in trange(num_runs):
    model_folder = f"{result_folder}/PAN_SCD_CL/run_{i_run}"
    Path(model_folder).mkdir(parents=True, exist_ok=True)
    
    lr = LR[i_run]
    freeze_bert = FREEZE_BERT[i_run]
    warmup_ratio = WARMUP_RATIO[i_run]
    loss_fn = LOSS_FN[i_run]
    
    my_training_args.lr = lr
    my_training_args.freeze_bert = int(freeze_bert) if freeze_bert != True else freeze_bert # json doesn't work with np.int64
    my_training_args.warmup_ratio = warmup_ratio
    my_training_args.loss_fn = loss_fn
    
    # use wandb to track experiments
    wconfig = {}
    wconfig['lr'] = lr
    wconfig['freeze_bert'] = freeze_bert
    wconfig['warmup_ratio'] = warmup_ratio
    wconfig['loss_fn'] = loss_fn
    

    run = wandb.init(project="PAN SCD with contrastive learning", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run {i_run}',
                     reinit=True)

    model = init_model(my_training_args)
    model = freeze_model(model, freeze_bert)
    
    wandb.watch(model, log="all", log_freq=1000, log_graph=True)

    optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad==True], lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*num_epochs*num_training_steps),
                            num_training_steps=num_epochs*num_training_steps)

    # start training and logging
    best_metric = 0.0
    previous_metric = 0.0
    early_stop_counter = 0
    df = pd.DataFrame(columns=['global_step'])
    pbar = trange(num_epochs*num_training_steps, leave=False)
    for i_epoch in range(num_epochs):
        model.train()
        for i_step in range(num_training_steps):

            batch = next(cycle_train_loader)
            optimizer.zero_grad()
            output = model(**batch)
            loss = output['loss']
            loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        evaluation = do_eval(model, cycle_val_loader, num_valid_steps)
        wandb.log(evaluation, step=pbar.n)
        evaluation.update({'global_step':pbar.n})
        df = df.append(evaluation, ignore_index=True)
        
        # save best model
        if save_best_only and (best_metric < evaluation[monitering_metric]):
            best_metric = evaluation[monitering_metric]
            wandb.log({f'best_{monitering_metric}': best_metric})
            torch.save(model.state_dict(), f"{model_folder}/pytorch_model.bin")
            torch.save(optimizer.state_dict(), f"{model_folder}/optimizer.pt")
            torch.save(scheduler.state_dict(), f"{model_folder}/scheduler.pt")
        
        if previous_metric < evaluation[monitering_metric]:
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print(f'Early stopped run {i_run}!')
                pbar.close()                
                break
        previous_metric = evaluation[monitering_metric]
    # if not save best, save the last
    if not save_best_only:
        torch.save(model.state_dict(), f"{model_folder}/pytorch_model.bin")
        torch.save(optimizer.state_dict(), f"{model_folder}/optimizer.pt")
        torch.save(scheduler.state_dict(), f"{model_folder}/scheduler.pt")
    with open(f"{model_folder}/training_args.json", "w") as outfile:
        json.dump(asdict(my_training_args), outfile)
    df.to_csv(f"{model_folder}/evaluation.csv", index=False)
    run.finish()


  0%|          | 0/36 [00:00<?, ?it/s]

wandb: Currently logged in as: cpuyyp (fsu-dsc-cil). Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


  0%|          | 0/28000 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

...

best_f1,▁█
f1,▆█▆▄▅▅▃▃▂▂▁▁▁
best_f1,0.7
f1,0.64306


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


  0%|          | 0/28000 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]